In [1]:
!pip install -q transformers rich torchmetrics sklearn pytorch-lightning sentence-transformers

In [2]:
import pandas as pd
import numpy as np
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3
import dask.bag as db
import json
import zipfile
from sklearn.preprocessing import MultiLabelBinarizer
import csv
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig
from sklearn.model_selection import train_test_split
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import RichProgressBar
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from torchmetrics import F1
from torchmetrics.functional import accuracy
from transformers import RobertaTokenizer, RobertaModel, AutoTokenizer, AutoModel
from transformers import AutoModel, AdamW, get_cosine_schedule_with_warmup
import math
from torchmetrics.functional.classification import auroc
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer
import pickle



In [3]:
!mkdir dataset

In [4]:
raw_json_data = "arxiv-metadata-oai-snapshot-json.zip"
pickled_dataset = 'dataset_pickle.pkl'
pickled_dataset_onehot = 'dataset_multilabel.pkl'

In [5]:
def __iter__(self): return 0

client_c2b4ce19d76d4b7c87bdf6c8c84d662c = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='SECRET',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

streaming_body_1 = client_c2b4ce19d76d4b7c87bdf6c8c84d662c.get_object(Bucket='SECRET', Key='arxiv-metadata-oai-snapshot.json.zip')['Body']

def save_to_cloud(key, file):
    client_c2b4ce19d76d4b7c87bdf6c8c84d662c.upload_file(Bucket='SECRET', Key=f'snapshot-{file}', Filename=file)

    
def get_file(key, filename):
    client_c2b4ce19d76d4b7c87bdf6c8c84d662c.download_file(Bucket='SECRET', Key=key, Filename=filename)

In [6]:
get_file('snapshot-uni_df.pkl', 'uni_df.pkl')

In [7]:
df_onehot = pd.read_pickle('uni_df.pkl')

In [12]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

pipeline = Pipeline([
    ('vct', TfidfVectorizer(strip_accents='unicode', stop_words='english', max_df=0.9, sublinear_tf=True)),
    ('clf', OneVsRestClassifier(LogisticRegression(C=1.1, multi_class='multinomial', max_iter=300, verbose=1, n_jobs=-1)))
])

In [13]:
RANDOM_SEED = 1337
df_train, df_test = train_test_split(df_onehot, test_size=0.1, random_state=RANDOM_SEED, shuffle=True)
df_train, df_validation = train_test_split(df_train, test_size=0.2, random_state=RANDOM_SEED, shuffle=True)
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_validation = df_validation.reset_index(drop=True)

In [14]:
X_train = df_train["abstract"]
y_train = df_train.loc[:, ~df_train.columns.isin(['abstract', 'embeddings'])]
X_test = df_test["abstract"]
y_test = df_test.loc[:, ~df_test.columns.isin(['abstract', 'embeddings'])]

In [15]:
%%time
pipeline.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =       296742     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.84497D+04    |proj g|=  4.74310D+04


 This problem is unconstrained.



At iterate   50    f=  3.14828D+03    |proj g|=  4.36771D+00

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
*****     82     93      1     0     0   5.264D-02   3.148D+03
  F =   3147.9208831787832     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   17.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


...

 This problem is unconstrained.



At iterate   50    f=  3.99947D+03    |proj g|=  1.68006D+00

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
*****     81     96      1     0     0   4.411D-02   3.999D+03
  F =   3999.0650559962646     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   18.4s finished


CPU times: user 23.3 s, sys: 12.9 s, total: 36.2 s
Wall time: 41min 21s


Pipeline(steps=[('vct',
                 TfidfVectorizer(max_df=0.9, stop_words='english',
                                 strip_accents='unicode', sublinear_tf=True)),
                ('clf',
                 OneVsRestClassifier(estimator=LogisticRegression(C=1.1,
                                                                  max_iter=300,
                                                                  multi_class='multinomial',
                                                                  n_jobs=-1,
                                                                  verbose=1)))])

In [16]:
X_test_preds_proba = pipeline.predict_proba(X_test)

In [17]:
def predict(tresh = 0.5, preds_proba = X_test_preds_proba, y_test = y_test):
    y_pred = [[int(val >= tresh) for val in pred] for pred in preds_proba]
    y_true = y_test
    print(classification_report(y_true, y_pred))
    print("Acc score: ", accuracy_score(y_true, y_pred))

In [18]:
predict()

/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.70      0.48      0.57       218
           1       0.88      0.52      0.65       192
           2       0.76      0.54      0.63       199
           3       0.82      0.55      0.66       235
           4       0.76      0.34      0.47       197
           5       0.77      0.57      0.65       285
           6       0.60      0.18      0.28       179
           7       0.75      0.39      0.52       320
           8       0.66      0.36      0.47       345
           9       0.38      0.02      0.04       145
          10       0.87      0.52      0.65       188
          11       0.78      0.48      0.59       281
          12       0.69      0.26      0.38       421
          13       0.74      0.47      0.58       260
          14       0.89      0.59      0.71       160
          15       0.52      0.16      0.24       500
          16       0.77      0.36      0.49        99
          17       0.62    

Acc score:  0.17118693496646253


In [19]:
predict(0.22)

/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.62      0.72      0.67       218
           1       0.73      0.70      0.72       192
           2       0.61      0.77      0.68       199
           3       0.72      0.76      0.74       235
           4       0.57      0.57      0.57       197
           5       0.62      0.76      0.68       285
           6       0.53      0.41      0.46       179
           7       0.53      0.60      0.56       320
           8       0.55      0.65      0.60       345
           9       0.30      0.09      0.14       145
          10       0.72      0.72      0.72       188
          11       0.64      0.71      0.67       281
          12       0.52      0.54      0.53       421
          13       0.57      0.74      0.65       260
          14       0.81      0.76      0.78       160
          15       0.40      0.42      0.41       500
          16       0.60      0.61      0.60        99
          17       0.47    

Acc score:  0.165864683581219


In [22]:
predict(0.15)

/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.52      0.86      0.65      3988
           1       0.60      0.82      0.69      1904
           2       0.66      0.88      0.75      1552
           3       0.59      0.83      0.69      1492
           4       0.65      0.85      0.74      1841
           5       0.69      0.85      0.76      1163
           6       0.50      0.77      0.61      1172
           7       0.50      0.78      0.61      1264
           8       0.59      0.81      0.68      1018
           9       0.37      0.68      0.48      1818
          10       0.43      0.69      0.53      1451
          11       0.43      0.69      0.53      1451
          12       0.59      0.86      0.70       883
          13       0.63      0.87      0.73      1136
          14       0.53      0.81      0.64       974
          15       0.35      0.60      0.44      1728
          16       0.56      0.80      0.66       727
          17       0.64    

Acc score:  0.16875854952122682


In [ ]:
predict(tresh = 0.2)

In [20]:
wml_credentials = {
    'apikey': 'SECRET',
    "url"         : "https://eu-gb.ml.cloud.ibm.com",
}

space_id = "SECRET"

In [21]:
from ibm_watson_machine_learning import APIClient

client = APIClient(wml_credentials)

In [24]:
sofware_spec_uid = client.software_specifications.get_id_by_name("runtime-22.1-py3.9")
metadata = {
            client.repository.ModelMetaNames.NAME: 'Scikit model',
            client.repository.ModelMetaNames.TYPE: 'scikit-learn_1.0',
            client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sofware_spec_uid
}
client.set.default_space(space_id)


'SUCCESS'

In [25]:
published_model = client.repository.store_model(
    model=pipeline,
    meta_props=metadata)

In [26]:
import json

published_model_uid = client.repository.get_model_uid(published_model)
model_details = client.repository.get_details(published_model_uid)
print(json.dumps(model_details, indent=2))

This method is deprecated, please use get_model_id()
{
  "entity": {
    "hybrid_pipeline_software_specs": [],
    "software_spec": {
      "id": "12b83a17-24d8-5082-900f-0ab31fbfd3cb",
      "name": "runtime-22.1-py3.9"
    },
    "type": "scikit-learn_1.0"
  },
  "metadata": {
    "created_at": "2022-06-12T16:30:36.405Z",
    "id": "20303ae1-3601-4952-9bea-ac5227837e05",
    "modified_at": "2022-06-12T16:31:05.958Z",
    "name": "Scikit model",
    "owner": "IBMid-6650023OBW",
    "resource_key": "f90b0811-6d8e-4c31-b1af-de8272569cd0",
    "space_id": "6f48a9aa-b319-450c-b6f1-bd3a789704d4"
  },
  "system": {
    "warnings": []
  }
}


In [27]:
metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "Deployment of external pytorch model",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

created_deployment = client.deployments.create(published_model_uid, meta_props=metadata)



#######################################################################################

Synchronous deployment creation for uid: '20303ae1-3601-4952-9bea-ac5227837e05' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.
.
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='5ee9c25d-c409-486f-a236-71bcbf72d2b2'
------------------------------------------------------------------------------------------------




In [61]:
created_deployment['metadata']['id']

'5ee9c25d-c409-486f-a236-71bcbf72d2b2'

In [62]:
deployment_id = created_deployment['metadata']['id']

In [73]:
# https://arxiv.org/abs/2008.04830
rzadca_paper = "Function as a Service (FaaS) permits cloud customers to deploy to cloud individual functions, in contrast to complete virtual machines or Linux containers. All major cloud providers offer FaaS products (Amazon Lambda, Google Cloud Functions, Azure Serverless); there are also popular open-source implementations (Apache OpenWhisk) with commercial offerings (Adobe I/O Runtime, IBM Cloud Functions). A new feature of FaaS is function composition: a function may (sequentially) call another function, which, in turn, may call yet another function - forming a chain of invocations. From the perspective of the infrastructure, a composed FaaS is less opaque than a virtual machine or a container. We show that this additional information enables the infrastructure to reduce the response latency. In particular, knowing the sequence of future invocations, the infrastructure can schedule these invocations along with environment preparation. We model resource management in FaaS as a scheduling problem combining (1) sequencing of invocations, (2) deploying execution environments on machines, and (3) allocating invocations to deployed environments. For each aspect, we propose heuristics. We explore their performance by simulation on a range of synthetic workloads. Our results show that if the setup times are long compared to invocation times, algorithms that use information about the composition of functions consistently outperform greedy, myopic algorithms, leading to significant decrease in response latency."

scoring_payload = {
client.deployments.ScoringMetaNames.INPUT_DATA: [{  
    "values": [
        [rzadca_paper]
    ]
 }]
}

In [68]:
predictions = client.deployments.score(deployment_id, scoring_payload)


In [69]:
categories = df_onehot.columns[1:].tolist()

In [71]:
categories

['astro-ph.CO',
 'astro-ph.EP',
 'astro-ph.GA',
 'astro-ph.HE',
 'astro-ph.IM',
 'astro-ph.SR',
 'cond-mat.dis-nn',
 'cond-mat.mes-hall',
 'cond-mat.mtrl-sci',
 'cond-mat.other',
 'cond-mat.quant-gas',
 'cond-mat.soft',
 'cond-mat.stat-mech',
 'cond-mat.str-el',
 'cond-mat.supr-con',
 'cs.AI',
 'cs.AR',
 'cs.CC',
 'cs.CE',
 'cs.CG',
 'cs.CL',
 'cs.CR',
 'cs.CV',
 'cs.CY',
 'cs.DB',
 'cs.DC',
 'cs.DL',
 'cs.DM',
 'cs.DS',
 'cs.ET',
 'cs.FL',
 'cs.GR',
 'cs.GT',
 'cs.HC',
 'cs.IR',
 'cs.IT',
 'cs.LG',
 'cs.LO',
 'cs.MA',
 'cs.MM',
 'cs.MS',
 'cs.NA',
 'cs.NE',
 'cs.NI',
 'cs.OH',
 'cs.PF',
 'cs.PL',
 'cs.RO',
 'cs.SC',
 'cs.SD',
 'cs.SE',
 'cs.SI',
 'cs.SY',
 'econ.EM',
 'econ.GN',
 'econ.TH',
 'eess.AS',
 'eess.IV',
 'eess.SP',
 'eess.SY',
 'gr-qc',
 'hep-ex',
 'hep-lat',
 'hep-ph',
 'hep-th',
 'math-ph',
 'math.AC',
 'math.AG',
 'math.AP',
 'math.AT',
 'math.CA',
 'math.CO',
 'math.CT',
 'math.CV',
 'math.DG',
 'math.DS',
 'math.FA',
 'math.GM',
 'math.GN',
 'math.GR',
 'math.GT',
 'ma

In [70]:
for cat, prob in zip(categories, predictions['predictions'][0]['values'][0][1]):
    if prob >= 0.2:
        print(cat)

cs.DC


In [98]:
#wml_python_function
def score_scikit_arxiv():
    from ibm_watson_machine_learning import APIClient

    wml_credentials = {
        'apikey': 'SECRET',
        "url"         : "https://eu-gb.ml.cloud.ibm.com",
    }

    space_id = "6f48a9aa-b319-450c-b6f1-bd3a789704d4"
    deployment_id = "5ee9c25d-c409-486f-a236-71bcbf72d2b2"
    client = APIClient(wml_credentials)
    client.set.default_space(space_id)
    categories = ['astro-ph.CO', 'astro-ph.EP', 'astro-ph.GA', 'astro-ph.HE', 'astro-ph.IM', 'astro-ph.SR', 'cond-mat.dis-nn', 'cond-mat.mes-hall', 'cond-mat.mtrl-sci', 'cond-mat.other', 'cond-mat.quant-gas', 'cond-mat.soft', 'cond-mat.stat-mech', 'cond-mat.str-el', 'cond-mat.supr-con', 'cs.AI', 'cs.AR', 'cs.CC', 'cs.CE', 'cs.CG', 'cs.CL', 'cs.CR', 'cs.CV', 'cs.CY', 'cs.DB', 'cs.DC', 'cs.DL', 'cs.DM', 'cs.DS', 'cs.ET', 'cs.FL', 'cs.GR', 'cs.GT', 'cs.HC', 'cs.IR', 'cs.IT', 'cs.LG', 'cs.LO', 'cs.MA', 'cs.MM', 'cs.MS', 'cs.NA', 'cs.NE', 'cs.NI', 'cs.OH', 'cs.PF', 'cs.PL', 'cs.RO', 'cs.SC', 'cs.SD', 'cs.SE', 'cs.SI', 'cs.SY', 'econ.EM', 'econ.GN', 'econ.TH', 'eess.AS', 'eess.IV', 'eess.SP', 'eess.SY', 'gr-qc', 'hep-ex', 'hep-lat', 'hep-ph', 'hep-th', 'math-ph', 'math.AC', 'math.AG', 'math.AP', 'math.AT', 'math.CA', 'math.CO', 'math.CT', 'math.CV', 'math.DG', 'math.DS', 'math.FA', 'math.GM', 'math.GN', 'math.GR', 'math.GT', 'math.HO', 'math.IT', 'math.KT', 'math.LO', 'math.MG', 'math.MP', 'math.NA', 'math.NT', 'math.OA', 'math.OC', 'math.PR', 'math.QA', 'math.RA', 'math.RT', 'math.SG', 'math.SP', 'math.ST', 'nlin.AO', 'nlin.CD', 'nlin.PS', 'nlin.SI', 'nucl-ex', 'nucl-th', 'physics.acc-ph', 'physics.ao-ph', 'physics.app-ph', 'physics.atm-clus', 'physics.atom-ph', 'physics.bio-ph', 'physics.chem-ph', 'physics.class-ph', 'physics.comp-ph', 'physics.data-an', 'physics.ed-ph', 'physics.flu-dyn', 'physics.gen-ph', 'physics.geo-ph', 'physics.hist-ph', 'physics.ins-det', 'physics.med-ph', 'physics.optics', 'physics.plasm-ph', 'physics.pop-ph', 'physics.soc-ph', 'physics.space-ph', 'q-bio.BM', 'q-bio.CB', 'q-bio.GN', 'q-bio.MN', 'q-bio.NC', 'q-bio.PE', 'q-bio.QM', 'q-bio.SC', 'q-bio.TO', 'q-fin.CP', 'q-fin.EC', 'q-fin.GN', 'q-fin.MF', 'q-fin.PM', 'q-fin.PR', 'q-fin.RM', 'q-fin.ST', 'q-fin.TR', 'quant-ph', 'stat.AP', 'stat.CO', 'stat.ME', 'stat.ML', 'stat.OT', 'stat.TH']
    threshold = 0.2
    
    def score( payload ):
        
        abstract_to_score = payload.get("input_data")[0].get("values")[0][0]
        scoring_payload = {
            client.deployments.ScoringMetaNames.INPUT_DATA: [{  
                "values": [
                    [abstract_to_score]
                ]
             }]
            }
        predictions = client.deployments.score(deployment_id, scoring_payload)
        predictions_labels = []
        predictions_proba = []
        
        for cat, prob in zip(categories, predictions['predictions'][0]['values'][0][1]):
            if prob >= threshold:
                predictions_labels.append(cat)
                predictions_proba.append(prob)

            
        # Score using the pre-defined model
        score_response = {
            'predictions': [{'fields': ['labels', 'probability'], 
                             'values': [[predictions_labels, predictions_proba]]
                            }]
        } 
        return score_response
    
    return score

In [99]:
input_data = { "input_data": [{
                                "values": [[ rzadca_paper ]]
                               }
                              ]
             }  
function_result = score_scikit_arxiv()( input_data )
print( function_result )

{'predictions': [{'fields': ['labels', 'probability'], 'values': [[['cs.DC'], [0.7681206986023242]]]}]}


In [100]:
software_spec_id =  client.software_specifications.get_id_by_name('runtime-22.1-py3.9')
print(software_spec_id)

12b83a17-24d8-5082-900f-0ab31fbfd3cb


In [101]:
function_meta_props = {
         client.repository.FunctionMetaNames.NAME: 'predict_categories_by_abstract_ovr_lr',
         client.repository.FunctionMetaNames.SOFTWARE_SPEC_UID: software_spec_id
         }

In [102]:
function_artifact = client.repository.store_function(meta_props=function_meta_props, function=score_scikit_arxiv)
function_uid = client.repository.get_function_id(function_artifact)
print("Function UID = " + function_uid)

Function UID = a50be72c-f76b-4184-b9ac-d1fc8d01e6cd


In [103]:
function_details = client.repository.get_details(function_uid)
from pprint import pprint
pprint(function_details)

{'entity': {'software_spec': {'id': '12b83a17-24d8-5082-900f-0ab31fbfd3cb',
                              'name': 'runtime-22.1-py3.9'},
            'type': 'python'},
 'metadata': {'created_at': '2022-06-12T17:51:43.219Z',
              'id': 'a50be72c-f76b-4184-b9ac-d1fc8d01e6cd',
              'modified_at': '2022-06-12T17:51:46.269Z',
              'name': 'predict_categories_by_abstract_ovr_lr',
              'owner': 'IBMid-6650023OBW',
              'space_id': '6f48a9aa-b319-450c-b6f1-bd3a789704d4'},
 'system': {'warnings': []}}


In [104]:
metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "predict_categories_by_abstract_ovr_lr",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

In [105]:
 deployment_details = client.deployments.create(function_uid, meta_props=metadata)




#######################################################################################

Synchronous deployment creation for uid: 'a50be72c-f76b-4184-b9ac-d1fc8d01e6cd' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.
..
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='38887e81-b43d-4f60-84fe-6c21c4317e6e'
------------------------------------------------------------------------------------------------




In [109]:
deployment_uid = deployment_details['metadata']['id']

In [118]:
job_payload = {
     client.deployments.ScoringMetaNames.INPUT_DATA: [{
         'values': [
             [rzadca_paper]
          ]
     }]
 }

In [119]:
client.deployments.score(deployment_uid, input_data)

{'predictions': [{'fields': ['labels', 'probability'],
   'values': [[['cs.DC'], [0.7681206986023242]]]}]}

In [125]:
def predict(abstract):
    job_payload = {
         client.deployments.ScoringMetaNames.INPUT_DATA: [{
             'values': [
                 [abstract]
              ]
         }]
     }
    return client.deployments.score(deployment_uid, job_payload)

In [126]:
# https://arxiv.org/abs/2203.14859
abstract = "FaaS (Function-as-a-Service) brought a fundamental shift into cloud computing: (persistent) virtual machines have been replaced with dynamically allocated resources, trading locality and statefulness for a pay-as-you-go model more suitable for varying and infrequent workloads. However, adapting services to function within the serverless paradigm while still fulfilling requirements is challenging. In this work, we introduce a design blueprint for creating complex serverless services and contribute a set of requirements for efficient and scalable FaaS computing. To showcase our approach, we focus on ZooKeeper, a centralized coordination service that offers a safe and wait-free consensus mechanism but requires a persistent allocation of computing resources that does not offer the flexibility needed to handle variable workloads. We design FaaSKeeper, the first coordination service built on serverless functions and cloud-native services. FaaSKeeper provides the same consistency guarantees and interface as ZooKeeper with a price model proportional to the activity in the system. In addition, we define synchronization primitives to extend the capabilities of scalable cloud storage ser- vices with consensus semantics needed for strong data consistency."
# real : cs.DC
predict(abstract)

{'predictions': [{'fields': ['labels', 'probability'],
   'values': [[['cs.DC'], [0.9062771029433222]]]}]}

In [127]:
# https://arxiv.org/abs/2206.04668
abstract = "Online action detection is the task of predicting the action as soon as it happens in a streaming video. A major challenge is that the model does not have access to the future and has to solely rely on the history, i.e., the frames observed so far, to make predictions. It is therefore important to accentuate parts of the history that are more informative to the prediction of the current frame. We present GateHUB, Gated History Unit with Background Suppression, that comprises a novel position-guided gated cross-attention mechanism to enhance or suppress parts of the history as per how informative they are for current frame prediction. GateHUB further proposes Future-augmented History (FaH) to make history features more informative by using subsequently observed frames when available. In a single unified framework, GateHUB integrates the transformer's ability of long-range temporal modeling and the recurrent model's capacity to selectively encode relevant information. GateHUB also introduces a background suppression objective to further mitigate false positive background frames that closely resemble the action frames. Extensive validation on three benchmark datasets, THUMOS, TVSeries, and HDD, demonstrates that GateHUB significantly outperforms all existing methods and is also more efficient than the existing best work. Furthermore, a flow-free version of GateHUB is able to achieve higher or close accuracy at 2.8x higher frame rate compared to all existing methods that require both RGB and optical flow information for prediction."
# real : cs.CV
predict(abstract)

{'predictions': [{'fields': ['labels', 'probability'],
   'values': [[['cs.CV'], [0.5055255144553442]]]}]}

In [131]:
# https://arxiv.org/abs/2206.04624
abstract = 'Pretrained language models (LMs) are susceptible to generate text with nonfactual information. In this work, we measure and improve the factual accuracy of large-scale LMs for open-ended text generation. We design the FactualityPrompts test set and metrics to measure the factuality of LM generations. Based on that, we study the factual accuracy of LMs with parameter sizes ranging from 126M to 530B. Interestingly, we find that larger LMs are more factual than smaller ones, although a previous study suggests that larger LMs can be less truthful in terms of misconceptions. In addition, popular sampling algorithms (e.g., top-p) in open-ended text generation can harm the factuality due to the "uniform randomness" introduced at every sampling step. We propose the factual-nucleus sampling algorithm that dynamically adapts the randomness to improve the factuality of generation while maintaining quality. Furthermore, we analyze the inefficiencies of the standard training method in learning correct associations between entities from factual text corpus (e.g., Wikipedia). We propose a factuality-enhanced training method that uses TopicPrefix for better awareness of facts and sentence completion as the training objective, which can vastly reduce the factual errors.'
# real : main cat: cs.CL , rest: cs.AI cs.CY cs.LG
predict(abstract)

{'predictions': [{'fields': ['labels', 'probability'],
   'values': [[['cs.CL', 'cs.LG'],
     [0.7536021828445011, 0.4138428771679431]]]}]}

In [132]:
# https://arxiv.org/abs/2206.04658
abstract = 'Despite recent progress in generative adversarial network(GAN)-based vocoders, where the model generates raw waveform conditioned on mel spectrogram, it is still challenging to synthesize high-fidelity audio for numerous speakers across varied recording environments. In this work, we present BigVGAN, a universal vocoder that generalizes well under various unseen conditions in zero-shot setting. We introduce periodic nonlinearities and anti-aliased representation into the generator, which brings the desired inductive bias for waveform synthesis and significantly improves audio quality. Based on our improved generator and the state-of-the-art discriminators, we train our GAN vocoder at the largest scale up to 112M parameters, which is unprecedented in the literature. In particular, we identify and address the training instabilities specific to such scale, while maintaining high-fidelity output without over-regularization. Our BigVGAN achieves the state-of-the-art zero-shot performance for various out-of-distribution scenarios, including new speakers, novel languages, singing voices, music and instrumental audio in unseen (even noisy) recording environments. We will release our code and model at: this https URL'
# real : main: cs.SD, rest: cs.CL, cs.LG, eess.AS
predict(abstract)

{'predictions': [{'fields': ['labels', 'probability'],
   'values': [[['cs.CV', 'cs.LG', 'cs.SD', 'eess.AS'],
     [0.22320128845614168,
      0.6862110249051099,
      0.9597862452691301,
      0.9617024296453474]]]}]}

In [133]:
# https://arxiv.org/abs/2008.04830
abstract = "Function as a Service (FaaS) permits cloud customers to deploy to cloud individual functions, in contrast to complete virtual machines or Linux containers. All major cloud providers offer FaaS products (Amazon Lambda, Google Cloud Functions, Azure Serverless); there are also popular open-source implementations (Apache OpenWhisk) with commercial offerings (Adobe I/O Runtime, IBM Cloud Functions). A new feature of FaaS is function composition: a function may (sequentially) call another function, which, in turn, may call yet another function - forming a chain of invocations. From the perspective of the infrastructure, a composed FaaS is less opaque than a virtual machine or a container. We show that this additional information enables the infrastructure to reduce the response latency. In particular, knowing the sequence of future invocations, the infrastructure can schedule these invocations along with environment preparation. We model resource management in FaaS as a scheduling problem combining (1) sequencing of invocations, (2) deploying execution environments on machines, and (3) allocating invocations to deployed environments. For each aspect, we propose heuristics. We explore their performance by simulation on a range of synthetic workloads. Our results show that if the setup times are long compared to invocation times, algorithms that use information about the composition of functions consistently outperform greedy, myopic algorithms, leading to significant decrease in response latency."
# real : cs.DC
predict(abstract)

{'predictions': [{'fields': ['labels', 'probability'],
   'values': [[['cs.DC'], [0.7681206986023242]]]}]}